In [1]:
import tkinter as tk
import cv2
from PIL import Image, ImageTk

import numpy as np
from mtcnn import MTCNN
import os
from keras_facenet import FaceNet
from sklearn.preprocessing import Normalizer, LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
from datetime import datetime
from openpyxl import Workbook

import time 

ModuleNotFoundError: No module named 'mtcnn'

In [ ]:
# raw_directory = "raw"
processed_directory = "processed"

# # check if raw directory exists
# if os.path.exists(raw_directory):
#     print(f"{raw_directory} already exists.")
# else:
#     print(f"{raw_directory} is not exist")
#     # create directory
#     os.mkdir(raw_directory)
#     print(f"{raw_directory} created successfully.")

# check if processed directory exists
if os.path.exists(processed_directory):
    print(f"{processed_directory} already exists.")
else:
    print(f"{processed_directory} is not exist")
    # create directory
    os.mkdir(processed_directory)
    print(f"{processed_directory} created successfully.")

processed already exists.


In [ ]:
# Crop face from Face Data _ AI1703 
data_path = 'F:\dataset\DEEEEEEPPPPP\Face Data'
for name in os.listdir(data_path):
    if name == '.DS_Store':
        continue
    elif os.path.exists("processed/" + name) == False:
        os.mkdir("processed/" + name)

detector = MTCNN()
data_path = 'F:\dataset\DEEEEEEPPPPP\Face Data'
num_of_frame = 0
for name in os.listdir(data_path):
    if name == '.DS_Store':
        continue
    name_path = os.path.join(data_path, name)
    for image_name in os.listdir(name_path):
        num_of_frame += 1
        if image_name == '.DS_Store':
            continue
        image_path = os.path.join(name_path, image_name)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        result = detector.detect_faces(image)

        print(result)

        if len(result) > 0:
            bounding_box = result[0]['box']
            keypoints = result[0]['keypoints']

            crop_img = image[bounding_box[1]:bounding_box[1] + bounding_box[3], bounding_box[0]:bounding_box[0]+bounding_box[2]]
            resized_image = cv2.resize(crop_img, (160, 160))
            save_img = cv2.cvtColor(resized_image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(f'processed/{name}/face {num_of_frame}.jpg', save_img)

1/1 [==============================] - 0s 83ms/step
[{'box': [1523, 578, 1208, 1641], 'confidence': 0.9999988079071045, 'keypoints': {'left_eye': (1897, 1167), 'right_eye': (2437, 1204), 'nose': (2177, 1444), 'mouth_left': (1919, 1776), 'mouth_right': (2377, 1814)}}]
1/1 [==============================] - 0s 19ms/step
[{'box': [1519, 607, 1190, 1649], 'confidence': 0.9999995231628418, 'keypoints': {'left_eye': (1884, 1176), 'right_eye': (2410, 1216), 'nose': (2156, 1447), 'mouth_left': (1906, 1804), 'mouth_right': (2346, 1835)}}]
1/1 [==============================] - 0s 24ms/step
[{'box': [1610, 533, 1163, 1631], 'confidence': 0.999994158744812, 'keypoints': {'left_eye': (2071, 1140), 'right_eye': (2604, 1219), 'nose': (2402, 1451), 'mouth_left': (2108, 1781), 'mouth_right': (2516, 1830)}}]
1/1 [==============================] - 0s 15ms/step
[{'box': [1759, 485, 1101, 1680], 'confidence': 0.9999862909317017, 'keypoints': {'left_eye': (2344, 1150), 'right_eye': (2776, 1224), 'nose': (2

In [ ]:
class App:
    def __init__(self, master):
        self.master = master
        self.master.title("Main Window")
        self.master.geometry('700x700')

        self.user_id = tk.StringVar()
        self.id_label = tk.Label(master, text='Register Student ID', font=('Times New Roman',10, 'bold')).place(x=50, y =150)
        self.enter_id = tk.Entry(master, textvariable=self.user_id, font=('calibre',10,'normal'),width = 30,).place(x=200, y =150)
        self.submit_bt = tk.Button(master, text='Submit', command=lambda: [self.submit_bt_push(), self.create_model()] , bd=4).place(x=420, y =150)
        self.track_bt = tk.Button(master, text='Track Face', command= self.open_subwindow, bd=4).place(x = 200, y = 235)
        self.quit_bt = tk.Button(master, text='Quit', command=master.destroy, bd = 4).place(x=200, y =270)




    def submit_bt_push(self):
        user_id = self.user_id.get()

        # NHẬP MÃ SV
        wrong_input = '\/:*?"<>|'
        while True:
            input_valid = True
            train_dir = user_id
            for char in train_dir:
                if char in wrong_input:
                    input_valid = False
                    print('Input contains invalid characters: \/:*?"<>|')
                    break
                    
            # phần trên là check input có được tạo thành folder hay không
            if input_valid == False:
                continue
            print("Input is valid.")
            # phần dưới là check folder đấy đã có chưa, có muốn thêm file/vid vào folder đấy hay không
            if os.path.exists("processed/" + train_dir):
                print(f"processed/{train_dir} already exists.")
                merge = input('input on the same folder(y/n):')
                if merge == 'y':
                    break
                else:
                    continue
            else:
                print(f"processed/{train_dir} is not exist.")
                # create directory
                os.mkdir("processed/" + train_dir)
                break

        # GET FACE TO PROCESS FILE
        cap = cv2.VideoCapture(0)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')
        face_num = 0
        interval = 5
        while True:
            ret, frame = cap.read()
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray_frame, 1.08, 7)

            for (x,y,w,h) in faces:
                crop_img = frame[y:y+w, x:x+w]
                roi_color = cv2.resize(crop_img, (160, 160))
                print(f'Capture img no {face_num}')
                if face_num % interval == 0:
                    cv2.imwrite(os.path.join(os.path.join("processed/", train_dir), f'frame {face_num}.jpg'), roi_color)
                face_num += 1

            cv2.imshow('cap', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()


    def create_model(self):
        faces = []
        Id = []
        IDUnique = []
        for subdir in os.listdir('processed'): #liệt kê mọi folder trong folder directory
            if os.path.isdir(os.path.join('processed', subdir)):
                IDUnique.append(subdir) # append mọi tên folder 
        for id in IDUnique:
            for facedir in os.listdir(os.path.join('processed', id)):
                face = cv2.imread(os.path.join('processed', id, facedir))
                faces.append(face)
                Id.append(id)

        # face embedder
        embedder = FaceNet()
        embeddings = embedder.embeddings(faces)

        # split train and test set
        X_train, X_test, y_train, y_test = train_test_split(embeddings, Id, test_size=.1, random_state=42)

        # 
        in_encoder = Normalizer(norm='l2')
        trainX = in_encoder.transform(X_train)
        testX = in_encoder.transform(X_test)
        out_encoder = LabelEncoder()
        out_encoder.fit(y_train)
        trainy = out_encoder.transform(y_train)
        testy = out_encoder.transform(y_test)
        model = SVC(kernel='linear')
        model.fit(trainX, trainy)

        # predict
        yhat_train = model.predict(trainX)
        yhat_test = model.predict(testX)
        # score
        score_train = accuracy_score(trainy, yhat_train)
        score_test = accuracy_score(testy, yhat_test)
        # summarize
        print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

        filename = 'svm_saved.sav'
        pickle.dump(model, open(filename, 'wb'))
        encoder_file_name = 'out_encoder.pkl'
        pickle.dump(out_encoder, open(encoder_file_name, 'wb'))
 
    def open_subwindow(self): # This part is for depoly model here
        vid = cv2.VideoCapture(0, cv2.CAP_DSHOW)
        vid.set(cv2.CAP_PROP_FPS, 20)

        detector = MTCNN()
        embedder = FaceNet()
        out_encoder = LabelEncoder()
        in_encoder = Normalizer(norm='l2')
        filename = 'svm_saved.sav'
        encoder_file_name = 'out_encoder.pkl'
        loaded_model = pickle.load(open(filename, 'rb'))
        loaded_out_encoder = pickle.load(open(encoder_file_name, 'rb'))
        wb = Workbook()
        sheet = wb.active
        sheet['A1'] = 'Name'
        sheet['B1'] = 'Time'
        # We need to check if camera
        # is opened previously or not
        if (vid.isOpened() == False): 
            print("Error reading video file")
        frame_width = int(vid.get(3))
        frame_height = int(vid.get(4))

        size = (frame_width, frame_height)

        #result = cv2.VideoWriter(f'final.avi', cv2.VideoWriter_fourcc(*'MJPG'),10, size)
        interval = 5  #muốn cách bao nhiêu ảnh
        frame_count = 0 
        attended = set()
        while(True):
            # Capture the video frame
            # by frame
            ret, frame = vid.read()
            if not ret:
                # If the frame was not successfully read, break out of the loop
                break

            #result.write(frame)
            # Display the resulting frame
            # cv2.imshow('frame', frame) 
            if frame_count % interval == 0:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                result = detector.detect_faces(image)
                if len(result) > 0:
                    for each_result in result:
                        bounding_box = each_result['box']
                        keypoints = each_result['keypoints']
                        crop_img = image[bounding_box[1]:bounding_box[1] + bounding_box[3], bounding_box[0]:bounding_box[0]+bounding_box[2]]
                        resized_image = cv2.resize(crop_img, (160, 160))
                        # save_img = cv2.cvtColor(resized_image, cv2.COLOR_RGB2BGR)
                        embeddings = embedder.embeddings([resized_image]) #########################
                        testX = in_encoder.transform(embeddings)
                        y_pred_encoded = loaded_model.predict(testX)
                        y_pred = loaded_out_encoder.inverse_transform(y_pred_encoded)
                        time_now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                        reg_frame = cv2.rectangle(image, (bounding_box[0], bounding_box[1]), (bounding_box[0]+bounding_box[2], bounding_box[1]+bounding_box[3]), (255,0,0), 2)
                        reg_frame = cv2.cvtColor(reg_frame, cv2.COLOR_BGR2RGB)
                        cv2.putText(reg_frame, text=str(y_pred[0]), fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=2, thickness=3, org=(bounding_box[0], bounding_box[1]-10), color=(0,255,0))
                        if y_pred[0] in attended:
                            continue
                        else:
                            attended.add(y_pred[0])
                            sheet.append([y_pred[0], time_now])

                    

            cv2.imshow('frame', reg_frame) 
            
            frame_count += 1
            
            # the 'q' button is set as the
            # quitting button you may use any
            # desired button of your choice
            if cv2.waitKey(1) & 0xFF == ord('q'):
                wb.save(filename="attend list.xlsx")
                break

        # After the loop release the cap object
        vid.release()
        # Destroy all the windows
        cv2.destroyAllWindows()


# create the main window
root = tk.Tk()
app = App(root)
root.mainloop()



Input is valid.
processed/AN is not exist.
Capture img no 0
Capture img no 1
Capture img no 2
Capture img no 3
Capture img no 4
Capture img no 5
Capture img no 6
Capture img no 7
Capture img no 8
Capture img no 9
Capture img no 10
Capture img no 11
Capture img no 12
Capture img no 13
Capture img no 14
Capture img no 15
Capture img no 16
Capture img no 17
Capture img no 18
Capture img no 19
Capture img no 20
Capture img no 21
Capture img no 22
Capture img no 23
Capture img no 24
Capture img no 25
Capture img no 26
Capture img no 27
Capture img no 28
Capture img no 29
Capture img no 30
Capture img no 31
Capture img no 32
Capture img no 33
Capture img no 34
Capture img no 35
Capture img no 36
Capture img no 37
Capture img no 38
Capture img no 39
Capture img no 40
Capture img no 41
Capture img no 42
Capture img no 43
13/13 [==============================] - 6s 368ms/step
Accuracy: train=100.000, test=100.000
1/1 [==============================] - 0s 69ms/step


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_26120\3293531187.py", line 193, in open_subwindow
    wb.save(filename="attend list.xlsx")
  File "c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\workbook\workbook.py", line 386, in save
    save_workbook(self, filename)
  File "c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\writer\excel.py", line 291, in save_workbook
    archive = ZipFile(filename, 'w', ZIP_DEFLATED, allowZip64=True)
  File "c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\zipfile.py", line 1249, in __init__
    self.fp = io.open(file, filemode)
PermissionError: [Errno 13] Permission denied: 'attend list.xlsx'
